In [2]:
import numpy as np 
import matplotlib.pyplot as plt 
import seaborn as sns

In [3]:
#helper func for standardization and getting gradients in lasso Regression
def standard_scalar(X):
    mean = X.mean(0)
    stds = X.std(0)
    return (X-mean)/stds

def sign(x,first_ele_zero=False):
    signs = (-1)**(x<0)
    if first_ele_zero:
        signs[0] = 0
    return signs    

In [4]:
class RegularizedRegression:
        
    def _record_info(self, X, y, lam, intercept, standardize):
        
        # standardize 
        if standardize == True: 
            X = standard_scalar(X)
        
        # add intercept
        if intercept == False: 
            ones = np.ones(len(X)).reshape(len(X), 1) # column of ones 
            X = np.concatenate((ones, X), axis = 1) # concatenate
            
        # record values
        self.X = np.array(X)
        self.y = np.array(y)
        self.N, self.D = self.X.shape
        self.lam = lam
        
    def fit_ridge(self, X, y, lam = 0, intercept = False, standardize = True):
        
        # record data and dimensions
        self._record_info(X, y, lam, intercept, standardize)
        
        # estimate parameters
        XtX = np.dot(self.X.T, self.X)
        I_prime = np.eye(self.D)
        I_prime[0,0] = 0 
        XtX_plus_lam_inverse = np.linalg.inv(XtX + self.lam*I_prime)
        Xty = np.dot(self.X.T, self.y)
        self.beta_hats = np.dot(XtX_plus_lam_inverse, Xty)
        
        # get fitted values
        self.y_hat = np.dot(self.X, self.beta_hats)
        
        
    def fit_lasso(self, X, y, lam = 0, n_iters = 2000,
                  lr = 0.0001, intercept = False, standardize = True):

        # record data and dimensions
        self._record_info(X, y, lam, intercept, standardize)
        
        # estimate parameters
        beta_hats = np.random.randn(self.D)
        for i in range(n_iters):
            dL_dbeta = -self.X.T @ (self.y - (self.X @ beta_hats)) + self.lam*sign(beta_hats, True)
            beta_hats -= lr*dL_dbeta 
        self.beta_hats = beta_hats
        
        # get fitted values
        self.y_hat = np.dot(self.X, self.beta_hats)

In [5]:
from sklearn.datasets import load_diabetes
diab = load_diabetes()
X  = diab['data']
y = diab['target']

In [6]:
X

array([[ 0.03807591,  0.05068012,  0.06169621, ..., -0.00259226,
         0.01990749, -0.01764613],
       [-0.00188202, -0.04464164, -0.05147406, ..., -0.03949338,
        -0.06833155, -0.09220405],
       [ 0.08529891,  0.05068012,  0.04445121, ..., -0.00259226,
         0.00286131, -0.02593034],
       ...,
       [ 0.04170844,  0.05068012, -0.01590626, ..., -0.01107952,
        -0.04688253,  0.01549073],
       [-0.04547248, -0.04464164,  0.03906215, ...,  0.02655962,
         0.04452873, -0.02593034],
       [-0.04547248, -0.04464164, -0.0730303 , ..., -0.03949338,
        -0.00422151,  0.00306441]])

In [7]:
ridge_model = RegularizedRegression()
ridge_model.fit_ridge(X,y,lam=10)

In [8]:
lasso_model = RegularizedRegression()
lasso_model.fit_lasso(X,y,lam=10)

In [9]:
sns.set()

Formula 2

In [ ]:
class Regularized:
    def __init__(self,lr,n_iters):
        self.lr = lr
        self.n_iters = n_iters
        self.lambda_reg = 0.1
        self.w,self.b = None,None
        
    def fit_lasso(self,X,y):
        n_samples,n_features = X.shape 
        self.w = np.zeros(n_features)
        self.b = 0
        
        for _ in range(self.n_iters):
            pred = np.dot(X ,self.w )+ self.b 
            dw = (1/n_samples) * np.dot(X.T,(pred-y)) + self.lambda_reg * np.sign(self.w)
            db = (1/n_samples) * np.sum(pred-y)
            
            self.w -= self.lr * dw 
            self.b -= self.lr * db 

        
        self.prediction =  np.dot(X,self.w) + self.b 
        return self.w,self.b 
            
    def fit_ridge(self,X,y):
        n_samples,n_features = X.shape 
        self.w = np.zeros(n_features)
        self.b = 0
        
        for _ in range(self.n_iters):
            pred =np.dot(X ,self.w )+ self.b 
            dw = (1/n_samples) * np.dot(X.T,(pred-y)) + self.lambda_reg * self.w
            db = (1/n_samples) * np.sum(pred-y)
            
            self.w -= self.lr * dw 
            self.b -= self.lr * db 
            
        self.prediction =  X * self.w + self.b 
        return self.w,self.b    